## 광안대교 정체를 위한 외부 요소 추가
첫번째 시도: 정체가 발생했으면 하는 엣지 ID를 포함하는 경로를 원본 osm.passenger.rou.xml 파일의 edges에서 추출하여 flow 발생시키기

In [ ]:
# 시작 엣지 파일 불러오기
file = open('target.txt', 'r')
target_value = []
for line in file:
  target_value.append(line.replace('\n', ''))

target_value

['77488351#1',
 '511894544#1-AddedOnRampEdge',
 '37403456#1',
 '701216011#0-AddedOnRampEdge',
 '-37399344',
 '506755491',
 '164788465#0',
 '-795710147',
 '795710147',
 '382760171',
 '382760683',
 '382760546#1']

In [ ]:
# 원본 xml 파일에서 엣지 정보 추출
from lxml import etree

tree = etree.parse("osm.passenger.rou.xml")
root = tree.getroot()

datas = root.getchildren()
e_list = []

for d in datas:
  tmp = d.getchildren()
  for t in tmp:
    e_list.append(t.get("edges"))

# 중복 제거
e_list1 = set(e_list)
edge_list = list(e_list1)
edge_list = list(filter(None, edge_list))

filtered_list = []

for e in edge_list:
    elements = e.split()
    for t in target_value:
      if elements[0] == t:
          filtered_list.append(e)

filtered_list

['511894544#1-AddedOnRampEdge 511894544#1 511894544#2 -165500781#1 -655967778 655967777#0 655967777#1-AddedOnRampEdge 655967777#1 655967777#2 655967777#3 164788465#0 164788465#0-AddedOffRampEdge 164788465#1 474161216 474161219 474161217 37399344 801453631#0 801453631#2 795710147 801455980 801455981#0 801455981#2 801455982 165503746#0 165503746#1 165503746#2 165503746#3 165503746#4 165503746#5 165503746#6 165503748#0 165503748#1 165503748#2 165503748#3 165503748#5 165503748#7 165503748#9 165503748#10 165503748#11 165503748#12 165503748#13 165503748#14 37401540#0 37401540#1 -37397953#2',
 '382760546#1 203971519 382760164 77488333 164788472 511894544#1-AddedOnRampEdge 511894544#1 569372437 655967770 655967769 655967752#0 951590183 693695645 166118878#0 166118878#1 166118878#2 166118878#3 166118878#4 166118878#7 656241653#0 656241653#1 1181957264 801455983#0 801455983#1 801455983#2 801455983#3 801455983#4 801455983#5 801455983#6 1181957255 795710140 795710139#0 795710139#1 694638130#0 6946

In [ ]:
# 외부 요인 추가
# 네임스페이스 정의
namespaces = {
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}
# 루트 요소 생성 및 네임스페이스 설정
root = ET.Element('routes', {
    'xmlns:xsi': namespaces['xsi'],
    'xsi:noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/routes_file.xsd'
})

# vType
# vType = ET.SubElement(root, 'vType', {
#     'id': 'veh_passenger',
#     'vClass': 'passenger'
# })

# route 생성
nc_r = 0
route_id_list = []
for l in filtered_list:
  id_r_num = 'r_' + str(nc_r)
  route_id_list.append(id_r_num)

  route = ET.SubElement(root, 'route', {
            'id': id_r_num,
            'edges': l
        })
  nc_r += 1

# flow 생성
nc_f = 0
for n in range(10):
  for r in route_id_list:
    id_f_num = 'f_' + str(nc_f)
    flow = ET.SubElement(root, 'flow', {
              'id': id_f_num,
              'begin': '0.00',
              'route': r,
              'end': '3600',
              'vehsPerHour': '360'
          })
    nc_f += 1


# ElementTree 객체 생성
tree = ET.ElementTree(root)

# XML 문자열로 변환
xml_str = ET.tostring(root, encoding='utf-8')

# Sorting
dom = xml.dom.minidom.parseString(xml_str)
passenger_rou_xml = dom.toprettyxml(indent="  ")

# Save
with open('passenger.rou.xml', 'w', encoding='utf-8') as f:
    f.write(passenger_rou_xml)

두번째 시도: 원래 osm.passenger.rou.xml에서 광안대교 엣지를 포함하지 않는 경로만 추출하여 외부 요소로 추가하기


In [ ]:
# 광안대교 포함된 엣지 리스트
target_edge = ['382760156', '1180482804']

In [ ]:
# 원본 xml 파일에서 엣지 정보 추출 (passenger)
from lxml import etree

tree = etree.parse("osm.passenger.rou.xml")
root = tree.getroot()

datas = root.getchildren()
e_list = []

for d in datas:
  tmp = d.getchildren()
  for t in tmp:
    e_list.append(t.get("edges"))

# 중복 제거
unique_edges = list(set(e_list))

# 엣지 정보 필터링
filtered_list = []

for edges in unique_edges:
    if not any(target in edges.split() for target in target_edge):
        filtered_list.append(edges)

# 필터링된 엣지 리스트 출력
filtered_list

['-166192538#1 -166192538#0 794794067#7 794794067#8 794794067#9 794794067#10 794794067#11 794794068 776897193#1 776897193#2 776897193#3 776897192 776897191#0 776897191#1 776897191#2 794794064#0 794794064#1 794794064#2 794794064#3 794794064#4 794794064#5 794794064#6 794794064#7 26398657#1 26398657#2 26398657#3 26398657#4',
 '166138501#1 -166138495#0 -776897191#1 -776897191#0 -776897192 -776897194 -776897193#2 -776897193#1 -776897193#0 794794069#0 794794069#1 794794069#2 794794069#3 794794070#0 794794070#1 794794070#2 794794071#0 794794071#1 794794071#2 165503751#0 165503751#1 77488339#0 77488339#1 77488339#2 77488353 37395919#0 37395919#1 37395919#2 37395919#3 667383252#0 667383252#1 -666021451#4 -666021451#3 -666021451#2 -666021451#1 -666021451#0 -468202425 1009817534 511862474#1 511862476#0-AddedOnRampEdge 511862476#0 511890772 1009817530 1009752213 1009752217 1126003973',
 '779257458#2 667383250#0 391424543 391424544 626525991 1219806285#0 1219806285#1 1219806285#2 1219806285#3 12198

In [ ]:
# depart 추출
tree = etree.parse("osm.passenger.rou.xml")
root = tree.getroot()

d_list = []

for vehicle in root.findall(".//vehicle"):
    depart = vehicle.get("depart")
    if depart:
        d_list.append(depart)
d_list

['0.00',
 '0.65',
 '1.30',
 '1.95',
 '2.60',
 '3.26',
 '3.91',
 '4.56',
 '5.21',
 '5.86',
 '6.51',
 '7.16',
 '7.81',
 '8.47',
 '9.12',
 '9.77',
 '10.42',
 '11.07',
 '11.72',
 '12.37',
 '13.02',
 '13.68',
 '14.33',
 '14.98',
 '15.63',
 '16.28',
 '16.93',
 '17.58',
 '18.23',
 '18.88',
 '19.54',
 '20.19',
 '20.84',
 '21.49',
 '22.14',
 '22.79',
 '23.44',
 '24.09',
 '24.75',
 '25.40',
 '26.05',
 '26.70',
 '27.35',
 '28.00',
 '28.65',
 '29.30',
 '29.96',
 '30.61',
 '31.26',
 '31.91',
 '32.56',
 '33.21',
 '33.86',
 '34.51',
 '35.16',
 '35.82',
 '36.47',
 '37.12',
 '37.77',
 '38.42',
 '39.07',
 '39.72',
 '40.37',
 '41.03',
 '41.68',
 '42.33',
 '42.98',
 '43.63',
 '44.28',
 '44.93',
 '45.58',
 '46.24',
 '46.89',
 '47.54',
 '48.19',
 '48.84',
 '49.49',
 '50.14',
 '50.79',
 '51.44',
 '52.10',
 '52.75',
 '53.40',
 '54.05',
 '54.70',
 '55.35',
 '56.00',
 '56.65',
 '57.31',
 '57.96',
 '58.61',
 '59.26',
 '59.91',
 '60.56',
 '61.21',
 '61.86',
 '62.52',
 '63.17',
 '63.82',
 '64.47',
 '65.12',
 '65.7

In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

# 네임스페이스 정의
namespaces = {
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}
# 루트 요소 생성 및 네임스페이스 설정
root = ET.Element('routes', {
    'xmlns:xsi': namespaces['xsi'],
    'xsi:noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/routes_file.xsd'
})

# vType
vType = ET.SubElement(root, 'vType', {
    'id': 'veh_rou_passenger',
    'vClass': 'passenger'
})

# trip 부분
for nc, (fl, d) in enumerate(zip(filtered_list, d_list)):
    id = 'veh_rou' + str(nc)

    vehicle = ET.SubElement(root, 'vehicle', {
        'id': id,
        'type': 'veh_rou_passenger',
        'depart': str(d),  # depart 값을 문자열로 변환
        'departLane': 'best'
    })

    route = ET.SubElement(vehicle, 'route', {
        'edges': fl
    })

# ElementTree 객체 생성
tree = ET.ElementTree(root)

# XML 문자열로 변환
xml_str = ET.tostring(root, encoding='utf-8')

# XML 정렬 및 예쁘게 출력
dom = xml.dom.minidom.parseString(xml_str)
passenger_rou_xml = dom.toprettyxml(indent="  ")

# XML 저장
with open('passenger_rou.xml', 'w', encoding='utf-8') as f:
    f.write(passenger_rou_xml)


In [ ]:
# 원본 xml 파일에서 엣지 정보 추출 (bus)
tree = etree.parse("osm.bus.rou.xml")
root = tree.getroot()

In [ ]:
datas = root.getchildren()
e_list = []

for d in datas:
  tmp = d.getchildren()
  for t in tmp:
    e_list.append(t.get("edges"))

# 중복 제거
unique_edges = list(set(e_list))

# 엣지 정보 필터링
filtered_list = []

for edges in unique_edges:
    if not any(target in edges.split() for target in target_edge):
        filtered_list.append(edges)

# 필터링된 엣지 리스트 출력
filtered_list

['-165556872#0 -816394214#1 -816394214#0 -37399300#2 -37399300#0 794794085#1 794794085#2 794794085#3 794794085#4 794794085#5 794794085#6 794794085#7 166134567 -1126006071 1219806284#1 1219806284#2 1219806284#3 1219806284#4 1219806284#5 1219806284#6 1219806284#7 1219806284#9 1219806284#10 1219806284#11 1219806284#12 1219806284#13 1219806284#14 1219806284#15 1219806284#16 -37401445#5 -37401445#4 -37401445#3 -37401445#2 -37401445#1 -37401445#0 -801455982 -801455981#2 -801455981#0 -801455980 -795710147 -801453631#2 -801453631#1 -37399344 -474161217 -474161219 -474161216 77488351#0 77488351#1-AddedOnRampEdge 77488351#1 77488351#1-AddedOffRampEdge 511894544#0 511894544#1-AddedOnRampEdge 511894544#1 511894544#2 -165500781#1 -655967778 -655967776 37402102 -754700577',
 '-702286343#1 -702286343#0 1219806284#24 1219806284#24-AddedOffRampEdge 1219806284#25 795710140 795710139#0 795710139#1 694638130#0 694638130#1 694638130#2 694638131#1 77488343#0 77488343#1 77488343#2 77488343#3 77488343#4 77488

In [ ]:
# depart 추출
d_list = []

for vehicle in root.findall(".//vehicle"):
    depart = vehicle.get("depart")
    if depart:
        d_list.append(depart)
d_list

['0.00',
 '1.95',
 '3.91',
 '5.86',
 '7.81',
 '9.77',
 '11.72',
 '13.68',
 '15.63',
 '17.58',
 '19.54',
 '21.49',
 '23.44',
 '25.40',
 '27.35',
 '29.30',
 '31.26',
 '33.21',
 '35.16',
 '37.12',
 '41.03',
 '42.98',
 '44.93',
 '46.89',
 '48.84',
 '50.79',
 '52.75',
 '54.70',
 '56.65',
 '58.61',
 '60.56',
 '62.52',
 '64.47',
 '66.42',
 '68.38',
 '70.33',
 '72.28',
 '74.24',
 '76.19',
 '78.14',
 '80.10',
 '82.05',
 '84.00',
 '85.96',
 '87.91',
 '89.87',
 '91.82',
 '93.77',
 '95.73',
 '97.68',
 '99.63',
 '101.59',
 '103.54',
 '105.49',
 '107.45',
 '109.40',
 '111.36',
 '113.31',
 '115.26',
 '117.22',
 '119.17',
 '121.12',
 '123.08',
 '125.03',
 '126.98',
 '128.94',
 '130.89',
 '132.84',
 '134.80',
 '136.75',
 '138.71',
 '140.66',
 '142.61',
 '144.57',
 '146.52',
 '148.47',
 '150.43',
 '152.38',
 '154.33',
 '156.29',
 '158.24',
 '160.20',
 '162.15',
 '164.10',
 '166.06',
 '168.01',
 '169.96',
 '171.92',
 '173.87',
 '175.82',
 '177.78',
 '179.73',
 '181.68',
 '183.64',
 '185.59',
 '187.55',
 

In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

# 네임스페이스 정의
namespaces = {
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}
# 루트 요소 생성 및 네임스페이스 설정
root = ET.Element('routes', {
    'xmlns:xsi': namespaces['xsi'],
    'xsi:noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/routes_file.xsd'
})

# vType
vType = ET.SubElement(root, 'vType', {
    'id': 'bus_bus_rou',
    'vClass': 'bus'
})

# trip 부분
for nc, (fl, d) in enumerate(zip(filtered_list, d_list)):
    id = 'bus_rou' + str(nc)

    vehicle = ET.SubElement(root, 'vehicle', {
        'id': id,
        'type': 'bus_bus_rou',
        'depart': str(d),  # depart 값을 문자열로 변환
        'departLane': 'best'
    })

    route = ET.SubElement(vehicle, 'route', {
        'edges': fl
    })

# ElementTree 객체 생성
tree = ET.ElementTree(root)

# XML 문자열로 변환
xml_str = ET.tostring(root, encoding='utf-8')

# XML 정렬 및 예쁘게 출력
dom = xml.dom.minidom.parseString(xml_str)
bus_rou_xml = dom.toprettyxml(indent="  ")

# XML 저장
with open('bus_rou.xml', 'w', encoding='utf-8') as f:
    f.write(bus_rou_xml)


In [ ]:
# 원본 xml 파일에서 엣지 정보 추출 (truck)
tree = etree.parse("osm.truck.rou.xml")
root = tree.getroot()

In [ ]:
datas = root.getchildren()
e_list = []

for d in datas:
  tmp = d.getchildren()
  for t in tmp:
    e_list.append(t.get("edges"))

# 중복 제거
unique_edges = list(set(e_list))

# 엣지 정보 필터링
filtered_list = []

for edges in unique_edges:
    if not any(target in edges.split() for target in target_edge):
        filtered_list.append(edges)

# 필터링된 엣지 리스트 출력
filtered_list

['795055164#1 -795055161#0 -37400598#1 -37400598#0 -165503748#11 -165503748#10 -165503748#9 -165503748#8 -165503748#6 -165503748#4 -165503748#2 -165503748#1 -165503748#0 -165503746#6 -165503746#5 -165503746#4 -165503746#3 -165503746#2 -165503746#1 -165503746#0 -801455982 -801455981#2 -801455981#0 -801455980 -795710147 -801453631#2 -801453631#1 -37399344 -474161217 -474161219 -474161216 77488351#0 77488351#1-AddedOnRampEdge 77488351#1 77488351#1-AddedOffRampEdge 511894544#0 511894544#1-AddedOnRampEdge 511894544#1 569372437 655967770 655967769 655967752#0 951590183 693695645 166118184#0 166118184#1 795694455 166118185#3',
 '-794688368#0 -215390613#1 -215390613#0 -166132735#2 -166132735#1 -166132735#0 -26398657#4 -26398657#3 -26398657#2 -26398657#1 1219806284#4 1219806284#5 1219806284#6 1219806284#7 1219806284#9 1219806284#10 1219806284#11 1219806284#12 1219806284#13 1219806284#14 1219806284#15 1219806284#16 1219806284#17 1219806284#18 1219806284#19 1219806284#20 1219806284#21 1219806284#

In [ ]:
# depart 추출
d_list = []

for vehicle in root.findall(".//vehicle"):
    depart = vehicle.get("depart")
    if depart:
        d_list.append(depart)
d_list

['0.00',
 '0.98',
 '1.95',
 '2.93',
 '3.91',
 '4.88',
 '5.86',
 '6.84',
 '7.81',
 '8.79',
 '9.77',
 '10.74',
 '11.72',
 '12.70',
 '13.68',
 '14.65',
 '15.63',
 '16.61',
 '17.58',
 '18.56',
 '20.51',
 '21.49',
 '22.47',
 '23.44',
 '24.42',
 '25.40',
 '26.37',
 '27.35',
 '28.33',
 '29.30',
 '30.28',
 '31.26',
 '32.23',
 '33.21',
 '34.19',
 '35.16',
 '36.14',
 '37.12',
 '38.10',
 '39.07',
 '40.05',
 '41.03',
 '42.00',
 '42.98',
 '43.96',
 '44.93',
 '45.91',
 '46.89',
 '47.86',
 '48.84',
 '49.82',
 '50.79',
 '51.77',
 '52.75',
 '53.72',
 '54.70',
 '55.68',
 '56.65',
 '57.63',
 '58.61',
 '59.58',
 '60.56',
 '61.54',
 '62.52',
 '63.49',
 '64.47',
 '65.45',
 '66.42',
 '67.40',
 '68.38',
 '69.35',
 '70.33',
 '71.31',
 '72.28',
 '73.26',
 '74.24',
 '75.21',
 '76.19',
 '77.17',
 '78.14',
 '79.12',
 '80.10',
 '81.07',
 '82.05',
 '83.03',
 '84.00',
 '84.98',
 '85.96',
 '86.94',
 '87.91',
 '88.89',
 '89.87',
 '90.84',
 '91.82',
 '92.80',
 '93.77',
 '94.75',
 '95.73',
 '96.70',
 '97.68',
 '98.66',
 

In [ ]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

# 네임스페이스 정의
namespaces = {
    'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
}
# 루트 요소 생성 및 네임스페이스 설정
root = ET.Element('routes', {
    'xmlns:xsi': namespaces['xsi'],
    'xsi:noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/routes_file.xsd'
})

# vType
vType = ET.SubElement(root, 'vType', {
    'id': 'truck_truck_rou',
    'vClass': 'truck'
})

# trip 부분
for nc, (fl, d) in enumerate(zip(filtered_list, d_list)):
    id = 'truck_rou' + str(nc)

    vehicle = ET.SubElement(root, 'vehicle', {
        'id': id,
        'type': 'truck_truck_rou',
        'depart': str(d),  # depart 값을 문자열로 변환
        'departLane': 'best'
    })

    route = ET.SubElement(vehicle, 'route', {
        'edges': fl
    })

# ElementTree 객체 생성
tree = ET.ElementTree(root)

# XML 문자열로 변환
xml_str = ET.tostring(root, encoding='utf-8')

# XML 정렬 및 예쁘게 출력
dom = xml.dom.minidom.parseString(xml_str)
truck_rou_xml = dom.toprettyxml(indent="  ")

# XML 저장
with open('truck_rou.xml', 'w', encoding='utf-8') as f:
    f.write(truck_rou_xml)
